<a href="https://colab.research.google.com/github/Biribbissolo/Prova/blob/main/Test_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q segmentation-models
!pip install tensorflow==2.9.3
!pip install h5py==2.10.0
!pip install plotly==5.3.1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import os

import nibabel as nib
import tensorflow as tf

import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from tqdm import tqdm

import skimage
from skimage.io import imread, imshow, imsave
from skimage.transform import resize

from tensorflow import keras
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras import metrics
from keras.callbacks import ReduceLROnPlateau

from segmentation_models import Unet
import math
from math import floor

import time
import random
from random import seed
from random import random

Useful functions

In [ ]:
def visualizer(segm, IMG_HEIGHT, IMG_WIDTH):     #Function useful for displaying segmentations

   segm = resize(segm,(IMG_HEIGHT,IMG_WIDTH,4), mode='constant', preserve_range=True)

   back = segm[:,:,0]
   kid = segm[:,:,1]
   tum = segm[:,:,2]
   cys = segm[:,:,3]

   back = (back == 1)
   kid = (kid == 1)
   tum = (tum == 1)
   cys = (cys == 1)

   all_segments = np.zeros((IMG_HEIGHT, IMG_WIDTH, 3))

   all_segments[back] = (1,0,0)
   all_segments[kid] = (0,1,0)
   all_segments[tum] = (0,0,1)
   all_segments[cys] = (1,1,0)

   return all_segments


def Test(dataset_path, model_path, output_path, first_subj, last_subj, view_dataset, save_prediction):

    range_subjects = np.arange(first_subj, last_subj + 1, 1)
    num_subj=len(range_subjects)

    IMG_HEIGHT= 512
    IMG_WIDTH= 512
    IMG_CHANNELS = 3
    NUM_CLASSES = 4

    model = load_model(model_path,custom_objects=None, compile=False)

    DSC_kidney_tumor_cyst = np.zeros(num_subj)
    DSC_tumor_cyst = np.zeros(num_subj)
    DSC_tumor = np.zeros(num_subj)

    RVD_kidney_tumor_cyst = np.zeros(num_subj)
    RVD_tumor_cyst = np.zeros(num_subj)
    RVD_tumor = np.zeros(num_subj)


    j=0
    for n in range (first_subj, last_subj + 1):

        case_path = dataset_path + "/case_" + str(n).zfill(5)
        volume = nib.load(os.path.join(case_path, "imaging.nii.gz"))
        segmentation = nib.load(os.path.join(case_path, "segmentation.nii.gz"))
        mask_affine = segmentation.affine

        print(volume.shape)
        n_slice,height,width = volume.shape
        X = np.zeros([n_slice, IMG_HEIGHT, IMG_WIDTH,IMG_CHANNELS], dtype=np.uint8)
        Y_automatic=np.zeros([n_slice, IMG_HEIGHT, IMG_WIDTH], dtype=np.uint8)

        volume = volume.slicer[0 : n_slice]
        segmentation = segmentation.slicer[0 : n_slice]

        Mask_supporto = segmentation.get_fdata().astype(np.uint8)
        Y_manual = to_categorical(Mask_supporto, num_classes=NUM_CLASSES, dtype='float32')
        Y_manual_stock=np.zeros([n_slice, IMG_HEIGHT, IMG_WIDTH, NUM_CLASSES], dtype=np.float32)


        Vol_supporto= volume.get_fdata().astype(np.int16)


        for k in range(0,n_slice):
          image=resize(Vol_supporto[k,:,:], (IMG_HEIGHT,IMG_WIDTH,1), mode='constant', preserve_range=True)
          X[k,:,:,:]=image

          if height!=IMG_HEIGHT or width!=IMG_WIDTH:
            Y_manual_stock[k,:,:,:]= resize(Y_manual[k,:,:,:],(IMG_HEIGHT,IMG_WIDTH,NUM_CLASSES), mode='constant', preserve_range=True)

          support = np.reshape(X[k,:,:,:],(1,IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS))
          softmax = model.predict(support)
          softmax = np.reshape(softmax,(IMG_HEIGHT,IMG_WIDTH,NUM_CLASSES))

          softmax=np.argmax(softmax,axis=2).astype(np.uint8)

          Y_automatic[k,:,:]=softmax

        if height!=IMG_HEIGHT or width!=IMG_WIDTH:
          Y_manual = Y_manual_stock

        # RVD Kidney or Tumor or Cyst
        auto = (Y_automatic==1).sum() + (Y_automatic==2).sum() + (Y_automatic==3).sum()
        manual = (Y_manual[:,:,:,1]==1).sum() + (Y_manual[:,:,:,2]==1).sum() + (Y_manual[:,:,:,3]==1).sum()
        RVD_kidney_tumor_cyst[j]=(auto - manual) / (manual)

        # RVD Tumor or Cyst
        auto = (Y_automatic==2).sum() + (Y_automatic==3).sum()
        manual = (Y_manual[:,:,:,2]==1).sum() + (Y_manual[:,:,:,3]==1).sum()
        RVD_tumor_cyst[j]=(auto - manual) / (manual)

        # RVD Tumor
        auto = (Y_automatic==2).sum()
        manual = (Y_manual[:,:,:,2]==1).sum()
        RVD_tumor[j]=(auto - manual) / (manual)

        # DSC Kidney or Tumor or Cyst
        support = np.argmax(Y_manual,axis=3).astype(np.uint8)
        intersection = (((Y_automatic == 1) | (Y_automatic == 2) | (Y_automatic == 3)) & ((support == 1) | (support == 2) | (support == 3))).sum()
        auto = (((Y_automatic == 1) | (Y_automatic == 2) | (Y_automatic == 3))).sum()
        manual = ((support == 1) | (support == 2) | (support == 3)).sum()
        DSC_kidney_tumor_cyst[j] = (2 * intersection) / (auto + manual)

        # DSC Tumor or Cyst
        intersection = (((Y_automatic == 2) | (Y_automatic == 3)) & ((support == 2) | (support == 3))).sum()
        auto = (((Y_automatic == 2) | (Y_automatic == 3))).sum()
        manual = ((support == 2) | (support == 3)).sum()
        DSC_tumor_cyst[j] = (2 * intersection) / (auto + manual)

        # RVD Tumor
        intersection = (((Y_automatic == 2)) & ((support == 2))).sum()
        auto = (((Y_automatic == 2))).sum()
        manual = ((support == 2)).sum()
        DSC_tumor[j] = (2 * intersection) / (auto + manual)

        j=j+1

        os.system('clear')

        if save_prediction == 'on':

          os.makedirs(output_path, exist_ok = True)
          Y_automatic_nii = nib.Nifti1Image(Y_automatic, affine=mask_affine)
          nib.save(Y_automatic_nii, output_path + '/case_' + str(n).zfill(5)+'.nii.gz')
          print('Subject saved successfully')

    print('Mean DSC for Kidney + Tumor + Cyst: ', round(np.mean(DSC_kidney_tumor_cyst) * 100,5),'% (+-', round(np.std(DSC_kidney_tumor_cyst) * 100,5),'%)')
    print('Mean DSC for Tumor + Cyst: ', round(np.mean(DSC_tumor_cyst) * 100,5),'% (+-', round(np.std(DSC_tumor_cyst) * 100,5),'%)')
    print('Mean DSC for Tumor: ', round(np.mean(DSC_tumor) * 100,5),'% (+-', round(np.std(DSC_tumor) * 100,5),'%)')
    print('\n')
    print('Mean RVD for Kidney + Tumor + Cyst: ', round(np.mean(RVD_kidney_tumor_cyst),5),'(+-', round(np.std(RVD_kidney_tumor_cyst),5),')')
    print('Mean RVD for Tumor + Cyst: ', round(np.mean(RVD_tumor_cyst),5),'(+-', round(np.std(RVD_tumor_cyst),5),')')
    print('Mean RVD for Tumor: ', round(np.mean(RVD_tumor),5),'(+-', round(np.std(RVD_tumor),5),')')


    if view_dataset == 'on':
      for n in range(0,X.shape[0]):
        fig = plt.figure(figsize=(15, 15))
        ax1 = fig.add_subplot(131)
        ax1.imshow(X[n,:,:,:]), ax1.set_title('TAC Image')

        ax2= fig.add_subplot(132)
        support = to_categorical(Y_automatic[n,:,:], num_classes=NUM_CLASSES, dtype='float32')
        ax2.imshow(visualizer(support,IMG_HEIGHT,IMG_WIDTH)), ax2.set_title('Automatic Segmentation')

        ax3= fig.add_subplot(133)
        ax3.imshow(visualizer(Y_manual[n,:,:,:],IMG_HEIGHT,IMG_WIDTH)), ax3.set_title('Manual Segmentation')

        plt.show()
        time.sleep(1.2)

In [ ]:
dataset_path = '/content/drive/your_dataset_path'
model_path='/content/drive/output_training_path/Epoch_05-Val_Loss0.00021.h5'
output_path = '/content/drive/save_predictions_path'

first_subj= 0          # example: start from subject 0
last_subj= 249         # example: end up in subject 249


```
Test(dataset_path, model_path, output_path, first_subj, last_subj, view_dataset, save_prediction)

```
*   If you want to see the predictions: view_dataset = 'on'
*   If you want to save the predictions in output_path: save_prediction = 'on'

In [ ]:
Test(dataset_path, model_path, output_path, first_subj, last_subj, 'on', 'off')

Predictions were created!